In [1]:
import os 
import pickle

print(os.listdir(os.path.join(os.getcwd(),"deterministic_long_policy", "deterministic_long_policy_g=8_a=8_t=25")))

p = os.path.join(os.getcwd(),"deterministic_long_policy",
                            "deterministic_long_policy_g=8_a=8_t=25","deterministic_long_policy_g=8_a=8_t=25.pkl")

pt = r"C:\\Users\\james\\MSCIActiveInferenceProject\\results\\deterministic_long_policy\\deterministic_long_policy_g=8_a=8_t=25\\deterministic_long_policy_g=8_a=8_t=25.pkl"
with open(p, "rb") as file:
    pkl_file = pickle.load(file)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\james\\MSCIActiveInferenceProject\\results\\deterministic_long_policy\\deterministic_long_policy_g=8_a=8_t=25'

In [38]:
import os
import yaml
import pathlib

def tuple_constructor(loader, node):
    return tuple(loader.construct_sequence(node))

yaml.add_constructor('tag:yaml.org,2002:python/tuple', tuple_constructor)

def yml_loader(path, exp_name, show, show_exp, version):
    with open(path, "r") as file:
        yaml_file = yaml.load(file, Loader=yaml.FullLoader)
    indep = yaml_file["parameters"]
    
    # indep_output = f"a={indep['alpha']}_g={indep['gamma']}_t={indep['time_steps']}"
    # dep_output = f"peak_memory={float(yaml_file['peak_memory_per_episode']):.2f}\ntime_per_episode={float(yaml_file['time_per_episode']):.2f}\npolicy_length_per_episode={float(yaml_file['policy_length_per_episode']):.2f}\nstate_space_coverage={float(yaml_file['state_space_coverage']):.2f}\nepisodes={yaml_file['episode_count']}"
    
    ### independent results
    alpha = indep['alpha']
    gamma = indep['gamma']
    time_steps = indep['time_steps']
    
    episode_count = f"{yaml_file['episode_count']}"
    
    experiment_res = yaml_file['episode_count'] == 0
    if experiment_res:
        indep_latex_row = f"{alpha} {gamma} {time_steps}"
        dep_latex_row = "FAIL"
        latex_row = f"{indep_latex_row} & & -- & -- & -- & -- & --& -- \\\\"
        print(latex_row)
        return {  "alpha": alpha, "gamma": gamma, "time_steps": time_steps, "peak_memory": 999999999, "state_space_coverage": 9999999,  "time_per_episode": 99999999, "pol_len_per_episode": 99999999,  "episode_count": 0, "latex_row": latex_row}
    

        
    
    ### Dependent Results
    peak_memory = f"{float(yaml_file['peak_memory_per_episode']/1024):.2f}" # megabytes
    time_per_episode = f"{float(yaml_file['time_per_episode']):.2f}"
    pol_len_per_episode = f"{float(yaml_file['policy_length_per_episode']):.2f}"
    state_space_coverage = f"{float(yaml_file['state_space_coverage']):.2f}"
    
    # upper_bound_memory = f"{(float(episode_count) * float(peak_memory)):.2f}"
    total_elapsed_time = f"{(float(episode_count) * (float(time_per_episode)/60)):.2f}" 
    goal_arrival = f"{yaml_file['goal_arrival']}"
    trap_arrival = f"{yaml_file['trap_arrival']}"
    
    if version == "performance":
        indep_latex_row = f"{alpha} {gamma} {time_steps}"
        dep_latex_row = f"{peak_memory} & {time_per_episode} & {pol_len_per_episode} & {state_space_coverage} & {episode_count} &  {total_elapsed_time}"
    if version == "verification":
        indep_latex_row = f"{alpha} & {gamma} & {time_steps}"
        dep_latex_row = f"{peak_memory} & {time_per_episode} & {pol_len_per_episode} & {state_space_coverage} & {goal_arrival} & {trap_arrival} &  {total_elapsed_time}"
    
    latex_row = f"{indep_latex_row} & & {dep_latex_row} \\\\"
    alpha_gamma_show = [(4, 4), (8, 8), (16, 16), (32, 32)]
    time_show = [150, 175, 200, 250]
    
    if version == "performance":
        
        if show_exp:
            print(exp_name)
        
        if show:
            if (alpha, gamma) in alpha_gamma_show and time_steps in time_show:
                
                print(latex_row)
                
        else:
            print(latex_row)
            
        return  {  "alpha": alpha, "gamma": gamma, "time_steps": time_steps, "peak_memory": float(peak_memory), "state_space_coverage": float(state_space_coverage),  "time_per_episode": float(time_per_episode), "pol_len_per_episode": float(pol_len_per_episode),  "episode_count": float(episode_count), "latex_row": latex_row }
    
    if version == "verification":
        if show_exp:
            print(exp_name)
        
        if show:
            if (alpha, gamma) in alpha_gamma_show and time_steps in time_show:
                
                print(latex_row)
                
        else:
            print(latex_row)
            
        return {  "alpha": alpha, "gamma": gamma, "time_steps": time_steps, "peak_memory": float(peak_memory), "state_space_coverage": float(state_space_coverage),  "time_per_episode": float(time_per_episode), "pol_len_per_episode": float(pol_len_per_episode),  "episode_count": float(episode_count), "goal_arrival": float(goal_arrival), "latex_row": latex_row}
        
    
    

In [39]:
# derive yaml data from results
import os
import yaml
import pathlib
import re
import math

aif_results = os.path.join(os.getcwd(),"AiF_real")

def extract_sort_key(filename):
    match = re.search(r'g=(\d+)_a=(\d+)_t=(\d+)', filename)  # Extract numbers
    if match:
        g, a, t = map(int, match.groups())  # Convert to integers
        return (g, a, t)  # Sort by g, then a, then t
    return (0, 0, 0)  # Default if no match found

def extract_experiments_clean(folder, exp, show, show_exp, version):
    """
    Given an experiment name, including the folder where it is, extract the results and present them
    :return: 
    """
    exp_folder = os.path.join(os.getcwd(), folder, exp)
    shortest_time_val = None
    highest_episodes_val = None
    lowest_state_coverage_val = None
    lowest_path_per_episode_val = None
    lowest_peak_memory_val = None
    shortest_time = 10 ** 100
    highest_episodes = 0
    lowest_state_coverage = 10 ** 100
    lowest_path_per_episode = 10 ** 100
    lowest_peak_memory = 10 ** 100
    if version == "verification":
        highest_goal_arrival = 0
        highest_goal_arrival_val =  None
    
    for exp_class in sorted(os.listdir(exp_folder)):
        exp_class_path = os.path.join(exp_folder,exp_class)
        
        print("This is:", exp_class)
        for dec_model in sorted(os.listdir(exp_class_path), key=extract_sort_key ):
            dec_model_path = os.path.join(exp_class_path,dec_model, dec_model + ".yml")
            res = yml_loader(dec_model_path,  exp_class_path + " " + dec_model, show, show_exp, version)
            if min(shortest_time, res["time_per_episode"]) == res["time_per_episode"]:
                shortest_time = min(shortest_time, res["time_per_episode"])
                shortest_time_val = res["latex_row"]
            if max(highest_episodes, res["episode_count"]) == res["episode_count"]:
                highest_episodes = max(highest_episodes, res["episode_count"])
                highest_episodes_val = res["latex_row"]
            if min(lowest_state_coverage, res["state_space_coverage"]) == res["state_space_coverage"]:
                lowest_state_coverage = min(lowest_state_coverage, res["state_space_coverage"])
                lowest_state_coverage_val = res["latex_row"]
            if min( lowest_path_per_episode, res["pol_len_per_episode"]) == res["pol_len_per_episode"] :
                lowest_path_per_episode  = min(lowest_path_per_episode , res["pol_len_per_episode"])
                lowest_path_per_episode_val = res["latex_row"]
            if min( lowest_peak_memory, res["peak_memory"]) == res["peak_memory"] :
                lowest_peak_memory  = min(lowest_peak_memory, res["peak_memory"])
                lowest_peak_memory_val = res["latex_row"]
            if version == "verification":
                if max(highest_goal_arrival, res["goal_arrival"]) == res["goal_arrival"] :
                    highest_goal_arrival  = min(lowest_peak_memory, res["goal_arrival"])
                    highest_goal_arrival_val = res["latex_row"]
            
        # reset things back to where they were before
        print("\n")
        print("lowest_peak_memory_val: ", lowest_peak_memory_val)
        print("Shortest Time Value: ", shortest_time_val)
        print("lowest_path_per_episode_val: ", lowest_path_per_episode_val)
        print("lowest_state_coverage_val: ", lowest_state_coverage_val)
        print("highest_episodes_val: ", highest_episodes_val)
        if version == "verification":
            print("highest_goal_arrival_val: ", highest_goal_arrival_val)
        print("\n")
            

        
        shortest_time_val = None
        highest_episodes_val = None
        lowest_state_coverage_val = None
        lowest_path_per_episode_val = None
        lowest_peak_memory_val = None
        shortest_time = math.inf
        highest_episodes = 0
        lowest_state_coverage = math.inf
        lowest_path_per_episode = math.inf
        lowest_peak_memory = math.inf
        if version == "verification":
            highest_goal_arrival = 0
            highest_goal_arrival_val =  None
            
            
            
    

# for exp_dir in os.listdir(aif_results):
#     exp_dir_path = os.path.join(aif_results,exp_dir)
#     for exp_class in os.listdir(exp_dir_path):
#         exp_class_path = os.path.join(exp_dir_path,exp_class)
#         for dec_model in os.listdir(exp_class_path):
#             dec_model_path = os.path.join(exp_class_path,dec_model, dec_model + ".yml")
#             yml_loader(dec_model_path,  exp_dir + " " + dec_model)
                
        



In [40]:
folder = os.path.join("AiF_real", "performance")

In [11]:
"""
5x5 experiment performance results
"""
exp_5x5 = "5_by_5_grid_experiment"

extract_experiments_clean(folder, exp_5x5, show=True, show_exp=False, version="performance")

This is: deterministic_long_policy
3222.901611328125
4015.4736328125
4287.16845703125
8 & 8 & 150 & & 4.19 & 17.78 & 8.22 & 0.27 & 18 &  5.33 \\
2580.458203125
3078.2076822916665
4269.419162326389
2970.6629464285716
7325.4560546875
4267.907552083333
4067.4371744791665
3612.5478515625
4429.6677734375
2425.5290178571427
3761.4481201171875
4467.21728515625
16 & 16 & 150 & & 4.36 & 21.26 & 10.00 & 0.33 & 14 &  4.96 \\
2624.8512369791665
3623.3831380208335
4531.058528645834
5679.97265625
3892.1209716796875
4748.840294471154
2633.7228190104165
3339.253363715278
4263.93828125
2667.7868303571427
3292.5157645089284
4401.475846354167
32 & 32 & 150 & & 4.30 & 20.86 & 9.73 & 0.31 & 15 &  5.21 \\


lowest_peak_memory_val:  16 & 16 & 75 & & 2.37 & 24.64 & 10.71 & 0.35 & 7 &  2.87 \\
Shortest Time Value:  8 & 8 & 100 & & 3.92 & 4.75 & 1.50 & 0.10 & 2 &  0.16 \\
lowest_path_per_episode_val:  8 & 8 & 100 & & 3.92 & 4.75 & 1.50 & 0.10 & 2 &  0.16 \\
lowest_state_coverage_val:  8 & 8 & 100 & & 3.92 & 4.7

In [12]:
"""
7x7 experiment performance results
"""
exp_7x7 = "7_by_7_grid_experiment"

extract_experiments_clean(folder, exp_7x7, show=True, show_exp=False, version="performance")


This is: stochastic_long_long_policy
10866.57470703125
16128.944580078125
18755.061089409723
8 & 8 & 175 & & 18.32 & 630.72 & 18.78 & 0.31 & 9 &  94.61 \\
9143.2041015625
14282.424560546875
19953.180013020832
8756.27294921875
17841.855143229168
17583.879991319445
7415.70361328125
17562.340494791668
21038.71316964286
11778.1962890625
14116.859375
18362.72189670139
16 & 16 & 175 & & 17.93 & 667.31 & 19.22 & 0.30 & 9 &  100.10 \\
10515.1748046875
12554.692578125
20538.0869140625
8065.48486328125
14891.119466145834
17013.38693576389
10300.664388020834
14460.68671875
18587.53515625
10489.337890625
14180.13427734375
19863.625418526786
32 & 32 & 175 & & 19.40 & 685.51 & 24.86 & 0.36 & 7 &  79.98 \\


lowest_peak_memory_val:  8 & 16 & 75 & & 7.24 & 444.64 & 14.50 & 0.22 & 4 &  29.64 \\
Shortest Time Value:  32 & 8 & 75 & & 8.55 & 321.73 & 16.00 & 0.22 & 2 &  10.72 \\
lowest_path_per_episode_val:  16 & 8 & 75 & & 8.93 & 1341.22 & 11.00 & 0.12 & 2 &  44.71 \\
lowest_state_coverage_val:  16 & 8 &

In [108]:
"""
9x9 experiment performance results
"""
exp_9x9 = "9_by_9_grid_experiment"

extract_experiments_clean(folder, exp_9x9, True, show_exp=False, version="performance")

This is: stochastic_long_long_policy
8 & 8 & 175 & & 42.61 & 1631.48 & 42.25 & 0.31 & 4 &  108.77 \\
8 & 8 & 250 & & 32.64 & 931.70 & 17.55 & 0.15 & 11 &  170.81 \\
16 & 16 & 175 & & 52.38 & 2607.58 & 57.00 & 0.46 & 2 &  86.92 \\
16 & 16 & 250 & & 80.81 & 5020.87 & 108.00 & 0.54 & 2 &  167.36 \\
32 & 32 & 175 & & 45.12 & 2477.89 & 42.75 & 0.32 & 4 &  165.19 \\
32 & 32 & 250 & & 76.36 & 3868.26 & 81.67 & 0.40 & 3 &  193.41 \\


lowest_peak_memory_val:  32 & 32 & 75 & & 17.81 & 725.99 & 18.00 & 0.20 & 2 &  24.20 \\
Shortest Time Value:  32 & 8 & 175 & & 32.70 & 477.13 & 9.00 & 0.12 & 2 &  15.90 \\
lowest_path_per_episode_val:  32 & 8 & 175 & & 32.70 & 477.13 & 9.00 & 0.12 & 2 &  15.90 \\
lowest_state_coverage_val:  32 & 8 & 175 & & 32.70 & 477.13 & 9.00 & 0.12 & 2 &  15.90 \\
highest_episodes_val:  8 & 8 & 250 & & 32.64 & 931.70 & 17.55 & 0.15 & 11 &  170.81 \\


This is: stochastic_long_policy
8 & 8 & 175 & & 35.74 & 216.83 & 34.75 & 0.31 & 4 &  14.46 \\
8 & 8 & 250 & & 33.93 & 157.83 &

In [41]:
"""
spiral maze experiment performance results
"""

maze_path_9x9 =  "spiral_maze_9_by_9_extension"
extract_experiments_clean(folder, maze_path_9x9, show=False, show_exp=False,version="performance")


This is: stochastic_long_long_policy
4 4 75 & & 7.82 & 1027.78 & 70.00 & 1.00 & 1 &  17.13 \\
4 4 125 & & 10.26 & 537.57 & 36.50 & 0.86 & 2 &  17.92 \\
4 4 175 & & 11.23 & 420.24 & 25.80 & 0.54 & 5 &  35.02 \\
8 4 75 & & 7.50 & 529.22 & 37.50 & 0.89 & 2 &  17.64 \\
8 4 125 & & 10.50 & 579.01 & 39.33 & 0.92 & 3 &  28.95 \\
8 4 175 & & 14.28 & 436.78 & 29.17 & 0.59 & 6 &  43.68 \\
16 4 75 & & 7.14 & 890.70 & 64.00 & 1.00 & 1 &  14.85 \\
16 4 125 & & 11.46 & 462.60 & 29.75 & 0.58 & 4 &  30.84 \\
16 4 175 & & 12.93 & 568.52 & 35.00 & 0.83 & 4 &  37.90 \\
32 4 75 & & 6.98 & 441.74 & 32.00 & 0.52 & 2 &  14.72 \\
32 4 125 & & 9.11 & 341.16 & 22.25 & 0.56 & 4 &  22.74 \\
32 4 175 & & 12.51 & 620.29 & 42.00 & 0.82 & 4 &  41.35 \\
4 8 75 & & -- & -- & -- & -- & --& -- \\
4 8 125 & & 11.56 & 426.26 & 31.00 & 0.59 & 4 &  28.42 \\
4 8 175 & & 12.11 & 463.46 & 31.40 & 0.60 & 5 &  38.62 \\
8 8 75 & & 6.03 & 299.97 & 21.50 & 0.52 & 2 &  10.00 \\
8 8 125 & & 9.38 & 543.60 & 33.50 & 0.70 & 2 &  18.12 \\

In [23]:
verification_folder = os.path.join("AiF_real", "verification")

"""
spiral maze experiment performance results
"""

dtmc_verification =  "dtmc_verification"
extract_experiments_clean(verification_folder, dtmc_verification, show=False, show_exp=False, version="performance")



This is: stochastic_short_policy
8 & 8 & 200 & & 14.76 & 286.53 & 13.86 & 0.21 & 14 &  66.86 \\
16 & 8 & 200 & & 20.35 & 278.23 & 9.38 & 0.16 & 21 &  97.38 \\
32 & 8 & 200 & & 20.19 & 319.34 & 11.00 & 0.18 & 18 &  95.80 \\
8 & 16 & 200 & & 21.31 & 322.34 & 11.65 & 0.20 & 17 &  91.33 \\
16 & 16 & 200 & & 17.17 & 302.85 & 10.32 & 0.18 & 19 &  95.90 \\
32 & 16 & 200 & & 20.92 & 474.09 & 11.88 & 0.20 & 16 &  126.42 \\
8 & 32 & 200 & & 21.07 & 438.34 & 11.65 & 0.20 & 17 &  124.20 \\
16 & 32 & 200 & & 20.18 & 464.90 & 9.95 & 0.17 & 19 &  147.22 \\
32 & 32 & 200 & & 20.97 & 344.71 & 10.53 & 0.18 & 19 &  109.16 \\


lowest_peak_memory_val:  8 & 8 & 200 & & 14.76 & 286.53 & 13.86 & 0.21 & 14 &  66.86 \\
Shortest Time Value:  16 & 8 & 200 & & 20.35 & 278.23 & 9.38 & 0.16 & 21 &  97.38 \\
lowest_path_per_episode_val:  16 & 8 & 200 & & 20.35 & 278.23 & 9.38 & 0.16 & 21 &  97.38 \\
lowest_state_coverage_val:  16 & 8 & 200 & & 20.35 & 278.23 & 9.38 & 0.16 & 21 &  97.38 \\
highest_episodes_val:  16 &

In [ ]:
from pymdp.control import construct_policies



In [11]:
from fractions import Fraction

example_frac = Fraction(0.333333333333333).limit_denominator()


print(example_frac)
print(type(example_frac))
print(str(example_frac))
print(type(str(example_frac)))


1/3
<class 'fractions.Fraction'>
1/3
<class 'str'>


In [12]:
from fractions import Fraction


def create_prism_file(unique_states, dtmc, prism_filename):
    # Create PRISM file
    

    cwd = os.getcwd()

    p = os.path.join(cwd, prism_filename)
    print(p)

    with open(p, "w") as f:
        f.write("dtmc\n\nmodule agent\n")
        f.write(f"    s : [0..{len(unique_states) - 1}] init 0;\n\n")

        # Convert transitions to PRISM format
        for from_state, to_states in dtmc.items():
            from_idx = from_state
            transitions_str = " + ".join(
                [f"{str(Fraction(prob).limit_denominator())} : (s'={to_state})" for to_state, prob in to_states.items()]
            )
            f.write(f"    [] s={from_idx} -> {transitions_str};\n")

        f.write("\nendmodule\n")


In [13]:
"""
DTMC PRISM file processing
"""
# 
# DTMC_yaml = os.path.join("AiF_real", "verification","dtmc_verification","stochastic_short_policy", "stochastic_short_policy_g=32_a=8_t=200", "stochastic_short_policy_g=32_a=8_t=200.yml")
# 
# DTMC_yaml_1 = os.path.join("AiF_real", "verification","dtmc_verification","stochastic_short_policy", "stochastic_short_policy_g=8_a=16_t=200", "stochastic_short_policy_g=8_a=16_t=200.yml")
exp_name = os.path.join("AiF_real", "verification","dtmc_verification","stochastic_short_policy")
def pm_dtmc_creation(exp_name):
    for exp in os.listdir(exp_name):
        yaml_file = os.path.join(exp_name, exp, exp + '.yml')
        with open(yaml_file, "r") as file_data:
            file_load = yaml.load(file_data, Loader=yaml.FullLoader)
        dtmc_load = file_load["dtmc"]
        print(dtmc_load)
        unique_locs = list(file_load["policy_per_start"].keys())
        prism_filename = os.path.join(exp_name, exp, exp + '_dtmc.pm')
        create_prism_file(unique_states=unique_locs, dtmc=dtmc_load, prism_filename=prism_filename)
# 
# with open(DTMC_yaml, "r") as file:
#     file_load = yaml.load(file, Loader=yaml.FullLoader)
# 
# dtmc_load = file_load["dtmc"]
# unique_locs = list(file_load["policy_per_start"].keys())
# prism_filename = "stochastic_short_policy_g=32_a=8_t=200_dtmc_model.pm"
# 
# create_prism_file(unique_states=unique_locs, dtmc=dtmc_load, prism_filename=prism_filename )


In [14]:
pm_dtmc_creation(exp_name=exp_name)

{0: {0: 0.3333333333333333, 1: 0.0, 7: 0.6666666666666666}, 1: {0: 0.2857142857142857, 1: 0.2857142857142857, 2: 0.2857142857142857, 8: 0.14285714285714285}, 2: {1: 0.3333333333333333, 2: 0.16666666666666666, 3: 0.16666666666666666, 9: 0.3333333333333333}, 3: {2: 0.6666666666666666, 3: 0.0, 4: 0.0, 10: 0.3333333333333333}, 4: {3: 0.6666666666666666, 4: 0.0, 5: 0.0, 11: 0.3333333333333333}, 5: {4: 0.75, 5: 0.0, 6: 0.0, 12: 0.25}, 6: {5: 0.75, 6: 0.25, 13: 0.0}, 7: {0: 0.0, 7: 0.4, 8: 0.2, 14: 0.4}, 8: {1: 0.2, 7: 0.0, 8: 0.4, 9: 0.0, 15: 0.4}, 9: {2: 0.0, 8: 0.0, 9: 0.0, 10: 0.5, 16: 0.5}, 10: {3: 0.0, 9: 0.0, 10: 0.3333333333333333, 11: 0.0, 17: 0.6666666666666666}, 11: {4: 0.0, 10: 0.0, 11: 0.4, 12: 0.0, 18: 0.6}, 12: {5: 0.2, 11: 0.2, 12: 0.2, 13: 0.4, 19: 0.0}, 13: {6: 0.6, 12: 0.2, 13: 0.2, 20: 0.0}, 14: {7: 0.0, 14: 0.3333333333333333, 15: 0.5, 21: 0.16666666666666666}, 15: {8: 0.16666666666666666, 14: 0.16666666666666666, 15: 0.0, 16: 0.16666666666666666, 22: 0.5}, 16: {9: 0.0, 1

In [79]:
# DTMC_yaml_1 = os.path.join("AiF_real", "verification","dtmc_verification","stochastic_short_policy", "stochastic_short_policy_g=8_a=16_t=200", "stochastic_short_policy_g=8_a=16_t=200.yml")
# 
# 
# with open(DTMC_yaml, "r") as file:
#     file_load_1 = yaml.load(file, Loader=yaml.FullLoader)
# 
# dtmc_load_1 = file_load_1["dtmc"]
# unique_locs_1 = list(file_load_1["policy_per_start"].keys())
# prism_filename_1 = "stochastic_short_policy_g=8_a=16_t=200_dtmc_model.pm"
# 
# create_prism_file(unique_states=unique_locs_1, dtmc=dtmc_load_1 , prism_filename=prism_filename_1 )

C:\Users\james\MSCIActiveInferenceProject\results\stochastic_short_policy_g=8_a=16_t=200_dtmc_model.pm


In [80]:
# DTMC_yaml_2 = os.path.join("AiF_real", "verification","dtmc_verification","stochastic_short_policy", "stochastic_short_policy_g=8_a=8_t=200", "stochastic_short_policy_g=8_a=8_t=200.yml")
# 
# 
# with open(DTMC_yaml_2 , "r") as file:
#     file_load_2 = yaml.load(file, Loader=yaml.FullLoader)
# 
# dtmc_load_2 = file_load_2["dtmc"]
# unique_locs_2 = list(file_load_2["policy_per_start"].keys())
# prism_filename_2 = "stochastic_short_policy_g=8_a=16_t=200_dtmc_model.pm"
# 
# create_prism_file(unique_states=unique_locs_2, dtmc=dtmc_load_2 , prism_filename=prism_filename_2 )

C:\Users\james\MSCIActiveInferenceProject\results\stochastic_short_policy_g=8_a=16_t=200_dtmc_model.pm


In [45]:
def derive_aif_prism_transitions(g_values):
    """
    Given the values of G, that correpsonds with each paths respective free erneregry values, I wanna derive the value based on these bath for each possible transition s to s'
    :param g_values: 
    :return: 
    """

C:\Users\james\MSCIActiveInferenceProject\results
